How to convert a Google sheet to a static marketing website

templates google doc?

templates google sheet?

In [ ]:
var util = require('util');
var GoogleSpreadsheet = require('google-spreadsheet');

function getTemplates(doc) {
    if(typeof doc === 'string') {
        doc = new GoogleSpreadsheet(doc);
    }
    return util.promisify(doc.getInfo)()
        .then(info => {
            console.log('Loaded doc: ' + info.title + ' by ' + info.author.email);
            return info;
        })
        .then(s => s.worksheets.reduce((obj, worksheet) => {
            var layoutTitle = worksheet.title.toLowerCase()
                .replace(/\s*layout|\s*data|\s*template/ig, '')
                .split(/\s+/ig).join('-');
        
            if( typeof obj[layoutTitle] == 'undefined' ) obj[layoutTitle] = {template: null, data: null};
            if( worksheet.title.toLowerCase().includes('data') ) obj[layoutTitle].data = worksheet;
            else obj[layoutTitle].template = worksheet;
            return obj;
        }, {}))
}
module.exports = getTemplates;


test templates google sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get templates from sheet', () => {
    
    it('should have at least one page', () => {
        return getTemplates(docsId)
            .then(t => {
                console.log(JSON.stringify(Object.keys(t).reduce((obj, cur) => {
                    obj[cur] = {};
                    obj[cur].template = ((t[cur].template || {}).title || '').substr(0, 1000);
                    obj[cur].data = (t[cur].data ? ('yes: ' + t[cur].data.rowCount) : 'no');
                    return obj;
                }, {}), null, 4))
                assert(Object.keys(t).length > 0, 'should have a page called template');
            })
    })
    
    
})


get google data sheet?

google sheet array objects?


In [ ]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
var util = require('util');

function getDataSheet(worksheet) {
    if(!worksheet) {
        return Promise.resolve(false);
    }
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => {
            var columns = [];
            return cells.reduce((acc, o, arr) => {
                if (o.row == 1) {
                    columns[o.col] = o.value;
                    return acc;
                }
                if (!acc[o.row - 2]) {
                    acc[o.row - 2] = {};
                }
                var html = md.render(o.value);
                // if all markdown did was insert a paragraph and line break, use value instead
                if(html.replace(/<\/?p>|\s+/ig, '') == o.value.replace(/<\/?p>|\s+/ig, '')) {
                    html = o.value;
                }
                acc[o.row - 2][columns[o.col]] = html;
                return acc;
            }, []);
        })
}
module.exports = getDataSheet;


test google data sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get data from data sheet', () => {
    
    it('should process at least one data sheet', () => {
        return getTemplates(docsId)
            .then(t => {
                var key = Object.keys(t).filter(k => t[k].data)[0];
                assert(typeof key != 'undefined', 'should have at least one data sheet');
                return getDataSheet(t[key].data);
            })
            .then(data => {
                console.log(data.length);
                console.log(data);
                assert(data.length > 0, 'should have data');
            })
    })
    
    
})


add a row of data to a google sheet?

add row data google sheet?


In [3]:
var path = require('path');
var util = require('util');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials;
if(fs.existsSync('./sheet to web-8ca5784e0b05.json')) {
    credentials = require('./sheet to web-8ca5784e0b05.json');
} else {
    credentials = require(path.join(PROFILE_PATH, '.credentials/sheet to web-8ca5784e0b05.json'));
}
var GoogleSpreadsheet = require('google-spreadsheet');

function addRow(docId, newRow) {
    var doc = new GoogleSpreadsheet(docId);
    
    return util.promisify(doc.useServiceAccountAuth)(credentials)
        .then(() => util.promisify(doc.getInfo)())
        .then(info => {
            console.log('Loaded doc: ' + info.title + ' by ' + info.author.email);
            return info;
        })
        .then(info => {
            sheet = info.worksheets[0];
            console.log('sheet #1: ' + sheet.title + ' ' + sheet.rowCount + 'x' + sheet.colCount);

            return util.promisify(doc.addRow)(1, newRow);
        });
}

module.exports = addRow;

[Function: addRow]

test google sheet add row?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var addRow = importer.import('add row data google sheet');

var docsId = '16b1Z0sQkYhtMUfP7qMRL3vRBPWQsRbSlntURkMqWGX0';

describe('add row data to google sheet', () => {
    
    it('should add a row', () => {
        return addRow(docsId, {
                timestamp: Date.now(),
                name: 'Test Name',
                email: 'test@test.com',
                subject: 'This is a test subject',
                message: 'This is a new message',
                responded: 0
            })
            .then(rows => {
                assert(typeof rows != 'undefined', 'should have added a row to the sheet');
            })
    })
    
    
})
